In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/frontend_final

/content/gdrive/MyDrive/frontend_final


In [ ]:
!pip install flask

In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken #insert you ngrok token here

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4304a1056bbba57c2b2dceac0348b9339778d3dab96681056acf533a9a1f4c39
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git #move this into frontend_final folder

Cloning into 'GroundingDINO'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 415 (delta 135), reused 125 (delta 121), pack-reused 239
Receiving objects: 100% (415/415), 12.85 MiB | 30.11 MiB/s, done.
Resolving deltas: 100% (213/213), done.


In [ ]:
! python -m pip install -e GroundingDINO

Obtaining file:///content/gdrive/MyDrive/frontend_final/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.6 MB/s eta 0:00:00
  Running setup.py develop for groundingdino


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-y4q3_1cb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-y4q3_1cb
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36587 sha256=00609144a01b42cf08987ad2b3806296517265f010d7a2b21d2d78d364ae847c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3xzrk7l3/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [ ]:
! pip install diffusers transformers accelerate scipy safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.4 MB/s eta 0:00:00


In [ ]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2023-12-12 05:13:59--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.111, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G  83.9MB/s    in 25s     

2023-12-12 05:14:24 (96.5 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/frontend_final/models.py')  # Replace with the actual path to your model.py file


In [ ]:
!ls

Coke_buttal.ipynb  GroundingDINO	__pycache__	      static
coke_buttal.py	   next_frontend.ipynb	sam_vit_h_4b8939.pth  templates


In [ ]:
#app.py
from flask import Flask, flash, request, redirect, url_for, render_template
import urllib.request
import os
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
import pickle
template_folder = '/content/gdrive/MyDrive/frontend_final/templates'

app = Flask(__name__, template_folder = template_folder)
run_with_ngrok(app)
UPLOAD_FOLDER = '/content/gdrive/MyDrive/frontend_final/static/uploads'

app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
Image_path = ""
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/')
def home():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        print("file is not in request files")
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        #print('upload_image filename: ' + filename)
        flash('Image successfully uploaded and displayed below')
        #runModel(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        Image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        with open('variable.pkl', 'wb') as file:
          pickle.dump(Image_path, file)
        %run model.py
        filename2 = 'annotated_image2.jpg'
        return render_template('index.html', filename=filename2)
    else:
        print("Invalid images")
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)

@app.route('/display/<filename>')
def display_image(filename):
    #print('display_image filename: ' + filename)
    return redirect(url_for('static', filename='final/' + filename), code=301)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://206a-34-125-84-219.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Dec/2023 09:42:00] "GET / HTTP/1.1" 200 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py", line 714, in run
    filename = file_finder(fpath)
  File "/usr/local/lib/python3.10/dist-packages/IPython/utils/path.py", line 109, in get_py_filename
    raise IOError('File `%r` not found.' % name)
OSError: File `'coke_buttal.py'` not found.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/li